In [5]:
#!/usr/bin/env python
# coding: utf-8

import os
import numpy as np
import xarray as xr
from shutil import copyfile

# root_dir = '/glade/u/home/hongli/work/russian/ens_forc_wrf2'
root_dir='/glade/u/home/hongli/scratch/2019_10_01gssha/ens_forc_wrf2'
wrf_raw_file = os.path.join(root_dir,'scripts/step1_asc_to_nc/WestWRF_2017120200_2018040723.nc')
dem_file = os.path.join(root_dir,'dem/step4_create_gridinfo/gridinfo.nc')

R = 6371.0 #km

outfolder = 'scripts/step2_prepare_complete_stnlist'
if not os.path.exists(os.path.join(root_dir, outfolder)):
    os.makedirs(os.path.join(root_dir, outfolder))
ofile = 'stnlist_slope.txt'
ofile_detail = 'more_details.txt'

# read dem files
print('read DEM data')
f_dem = xr.open_dataset(dem_file)
dem_lats = f_dem['latitude'].values[:] # (y,x)=(480, 1040)
dem_lons = f_dem['longitude'].values[:]
dem_elevs = f_dem['elev'].values[:]
dem_slp_ns = f_dem['gradient_n_s'].values[:]
dem_slp_es = f_dem['gradient_w_e'].values[:]
mask = f_dem['mask'].values[:]

# read WRF data
print('read WRF data')
f_wrf = xr.open_dataset(wrf_raw_file)
wrf_lats = f_wrf['lat'].values[:] #(21,18)
wrf_lons = f_wrf['lon'].values[:] 

# extract elev, slp_n, slp_e for all the wrf grids within the boundary
print('calculate')

# (1) repeat useful data to be (wrf_num, dem_num)
(wrf_ny,wrf_nx)=np.shape(wrf_lats)
wrf_num = wrf_ny*wrf_nx

(dem_ny,dem_nx)=np.shape(dem_lats)
dem_num = dem_ny*dem_nx

wrf_lat_arr = np.deg2rad(np.repeat(np.reshape(wrf_lats, (wrf_num,1)), dem_num, axis=1))  # (wrf_num, dem_num) 
wrf_lon_arr = np.deg2rad(np.repeat(np.reshape(wrf_lons, (wrf_num,1)), dem_num, axis=1))

dem_lat_arr = np.deg2rad(np.repeat(np.reshape(dem_lats, (1, dem_num)), wrf_num, axis=0))
dem_lon_arr = np.deg2rad(np.repeat(np.reshape(dem_lons, (1, dem_num)), wrf_num, axis=0))

# (3) calcualte Haversine distance (km)
a = (np.sin((wrf_lat_arr - dem_lat_arr)/2))**2 + np.cos(wrf_lat_arr)*np.cos(dem_lat_arr) * (np.sin((wrf_lon_arr - dem_lon_arr)/2))**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
d = R * c 

# (4) identify the minimum distance corresponding DEM grid index
d_min_index = np.argmin(d, axis=1) # index among dem_num. shape is (wrf_num).
d_min_y_id = d_min_index//dem_nx #row index among (dem_ny, dem_nx)
d_min_x_id = d_min_index%dem_nx #col index among (dem_ny, dem_nx)

# output
print('save')

wrf_num_valid =np.count_nonzero(mask)

f=open(os.path.join(root_dir, outfolder, ofile), 'w+')
f.write('NSITES\t'+str(wrf_num_valid)+'\n')
f.write('#STA_ID\tLAT\tLON\tELEV\tSLP_N\tSLP_E\tSTA_NAME\n') # add the last four columns for quality check 

for i in range(wrf_num):

    # identify wrf lat_lon index and values
    wrf_y_id = i//wrf_nx # global id
    wrf_x_id = i%wrf_nx
    
    if (mask[wrf_y_id,wrf_x_id]==1):
        
        wrf_lat = wrf_lats[wrf_y_id,wrf_x_id]
        wrf_lon = wrf_lons[wrf_y_id,wrf_x_id]
        sta_id = 'Row'+str('%03d' % wrf_y_id)+'Col'+str('%03d' % wrf_x_id) 
        stn_name = '"'+sta_id+'"'
#         print(wrf_y_id,wrf_x_id,wrf_lat,wrf_lon)
    
        # identify DEM and other info
        dem_y_id = d_min_y_id[i]
        dem_x_id = d_min_x_id[i]    
        elev = dem_elevs[dem_y_id, dem_x_id]
        slp_n = dem_slp_ns[dem_y_id, dem_x_id]
        slp_e = dem_slp_es[dem_y_id, dem_x_id]    

        f.write('%s, %f, %f, %d, %f, %f, %s\n' % (sta_id, wrf_lat, wrf_lon, elev, slp_n, slp_e, stn_name))
f.close()  

f=open(os.path.join(root_dir, outfolder, ofile_detail), 'w+')
f.write('NSITES\t'+str(wrf_num_valid)+'\n')
f.write('#STA_ID\tLAT\tLON\tELEV\tSLP_N\tSLP_E\tSTA_NAME\tDEM_y_id\tDEM_x_id\tDEM_lat\tDEM_lon\n') # add the last four columns for quality check 
for i in range(wrf_num):

    # identify wrf lat_lon index and values
    wrf_y_id = i//wrf_nx # global id 
    wrf_x_id = i%wrf_nx

    if (mask[wrf_y_id,wrf_x_id]==1):
        wrf_lat = wrf_lats[wrf_y_id,wrf_x_id]
        wrf_lon = wrf_lons[wrf_y_id,wrf_x_id]
        sta_id = 'Row'+str('%03d' % wrf_y_id)+'Col'+str('%03d' % wrf_x_id) 
        stn_name = '"'+sta_id+'"'

        # identify DEM and other info
        dem_y_id = d_min_y_id[i]
        dem_x_id = d_min_x_id[i]    
        elev = dem_elevs[dem_y_id, dem_x_id]
        slp_n = dem_slp_ns[dem_y_id, dem_x_id]
        slp_e = dem_slp_es[dem_y_id, dem_x_id]    
        dem_lat = dem_lats[dem_y_id, dem_x_id]
        dem_lon = dem_lons[dem_y_id, dem_x_id]

        f.write('%s, %f, %f, %d, %f, %f, %s, %d, %d, %f, %f\n' % (sta_id, wrf_lat, wrf_lon, elev, slp_n, slp_e, stn_name,
                                                                  dem_y_id, dem_x_id, dem_lat, dem_lon))
f.close()      

copyfile(os.path.join(root_dir, outfolder, ofile), os.path.join(root_dir, 'GMET_tpl/inputs/',ofile))

print('Done')

read DEM data
read WRF data
calculate
save
Done


In [6]:
wrf_num_valid

393

In [4]:
wrf_lons

array([[-123.50623 , -123.47137 , -123.4365  , -123.40164 , -123.366776,
        -123.33192 , -123.29705 , -123.262184, -123.227325, -123.19246 ,
        -123.15759 , -123.12273 , -123.08787 , -123.053   , -123.018135,
        -122.98327 , -122.94841 , -122.91354 , -122.87868 ],
       [-123.506424, -123.47155 , -123.43667 , -123.401794, -123.36692 ,
        -123.33204 , -123.297165, -123.26228 , -123.22741 , -123.19253 ,
        -123.15765 , -123.12277 , -123.0879  , -123.05302 , -123.01814 ,
        -122.98326 , -122.94839 , -122.913506, -122.87863 ],
       [-123.506615, -123.471725, -123.43684 , -123.40195 , -123.36706 ,
        -123.33217 , -123.29728 , -123.26239 , -123.22749 , -123.192604,
        -123.157715, -123.122826, -123.08793 , -123.05304 , -123.01815 ,
        -122.98326 , -122.948364, -122.913475, -122.878586],
       [-123.50681 , -123.47191 , -123.437004, -123.4021  , -123.367195,
        -123.3323  , -123.29739 , -123.26249 , -123.227585, -123.19268 ,
        -123.1